In [1]:
import warnings

from opticalglass.glass import get_filepath

import numpy as np
import pandas as pd

In [2]:
def xl_cols():
    caps_word = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
    cols = list(caps_word)
    
    for c1 in caps_word:
        for c2 in caps_word:
            cols.append(c1 + c2)
    return cols

In [3]:
def xl2df(fname):
    with warnings.catch_warnings():
        warnings.filterwarnings("ignore", category=UserWarning)
        worksheet_df = pd.read_excel(get_filepath(fname), header=None)
    worksheet_df.columns = xl_cols()[:worksheet_df.shape[1]]
    return worksheet_df

In [4]:
def df2gcat(xldf, r0, rk, c0, ck, **kwargs):
    name_col_offset = kwargs['name_col_offset']
    data_header = kwargs['data_header']
    glass_cat = xldf.loc[r0:rk, c0:ck]
    glass_cat.index = xldf.loc[r0:rk, name_col_offset]
    glass_cat.columns = xldf.loc[xldf.index[data_header], c0:ck]
    return glass_cat

In [5]:
def fill_in_nans(s):
    categories = {}
    data_header = []
    c_last = None
    r1 = rk = None
    for i, item in enumerate(s.items()):
        indx, c = item
        if not pd.isna(c):
            if c_last is not None:
                categories[c_last] = (r1, rk)
            c_last = c
            r1 = i, indx
            rk = i+1, indx
            data_header.append(c)
        else:
            rk = i+1, indx
            data_header.append(c_last)
    return data_header, categories

In [6]:
arrays = [
    ["T", "T", "T", "T", "T", "T", "T", "T"],
    ["bar", "bar", "bar", "bar", "foo", "foo", "foo", "foo"],
    [700, 600, 500, 400, 700, 600, 500, 400],
]

In [7]:
tuples = list(zip(*arrays))

In [8]:
tuples

[('T', 'bar', 700),
 ('T', 'bar', 600),
 ('T', 'bar', 500),
 ('T', 'bar', 400),
 ('T', 'foo', 700),
 ('T', 'foo', 600),
 ('T', 'foo', 500),
 ('T', 'foo', 400)]

In [9]:
index = pd.MultiIndex.from_tuples(tuples, names=["transmittance", "thickness", "Wavelength"])

In [10]:
s = pd.Series(np.random.randn(8), index=index)

In [11]:
s

transmittance  thickness  Wavelength
T              bar        700          -0.199416
                          600           0.639057
                          500          -1.735149
                          400           0.460107
               foo        700          -1.302486
                          600          -1.009524
                          500          -0.705347
                          400           2.212922
dtype: float64

In [12]:
s['T', 'foo', 400]

2.212921634060628

In [13]:
s['T', 'bar', 400]

0.4601069684774579

In [14]:
arrays = [
    ["nd", "vd", "T", "T", "T", "T", "T", "T", "T", "T"],
    [pd.NA, pd.NA, "bar", "bar", "bar", "bar", "foo", "foo", "foo", "foo"],
    [pd.NA, pd.NA, 700, 600, 500, 400, 700, 600, 500, 400],
]

In [15]:
tuples = list(zip(*arrays))

In [16]:
tuples

[('nd', <NA>, <NA>),
 ('vd', <NA>, <NA>),
 ('T', 'bar', 700),
 ('T', 'bar', 600),
 ('T', 'bar', 500),
 ('T', 'bar', 400),
 ('T', 'foo', 700),
 ('T', 'foo', 600),
 ('T', 'foo', 500),
 ('T', 'foo', 400)]

In [17]:
index = pd.MultiIndex.from_tuples(tuples, names=["lvl0", "lvl1", "lvl2"])

In [18]:
s = pd.Series(np.random.randn(10), index=index)

In [19]:
s

lvl0  lvl1  lvl2
nd    NaN   NaN    -0.273019
vd    NaN   NaN    -0.409866
T     bar   700     0.747683
            600     0.499083
            500     0.832136
            400    -0.856197
      foo   700     1.428590
            600     0.321159
            500    -2.269191
            400     0.911963
dtype: float64

In [20]:
s['nd', None, None]

-0.27301867989455453

In [21]:
df = pd.DataFrame(np.random.randn(3, 10), index=["BK7", "F2", "SF6"], columns=index)

In [22]:
df

lvl0        nd        vd         T                                          \
lvl1       NaN       NaN       bar                                     foo   
lvl2       NaN       NaN       700       600       500       400       700   
BK7  -0.911825 -0.623095  0.474865 -0.652418  0.766789  0.968327 -0.655125   
F2    0.269230 -2.191998 -1.100909 -1.048229  0.227380 -0.209260  0.217434   
SF6   1.910490  1.159418 -1.071696 -0.686889 -0.828628  0.223205  0.044836   

lvl0                                
lvl1                                
lvl2       600       500       400  
BK7  -0.277510  0.122415 -0.908927  
F2   -1.974429 -0.265917 -1.546434  
SF6   0.630721  0.377356  0.478245

In [23]:
df["nd"]

lvl1,NaN
lvl2,NaN
BK7,-0.911825
F2,0.269230
SF6,1.910490


In [24]:
df["T", "bar", 700]

BK7    0.474865
F2    -1.100909
SF6   -1.071696
Name: (T, bar, 700), dtype: float64

In [25]:
df["T", "bar"]

/var/folders/4z/p2jynsyn64qcg_j024rv_3vw0000gn/T/ipykernel_31136/2057812384.py:1: PerformanceWarning: indexing past lexsort depth may impact performance.
  df["T", "bar"]


lvl2,700,600,500,400
BK7,0.474865,-0.652418,0.766789,0.968327
F2,-1.100909,-1.048229,0.227380,-0.209260
SF6,-1.071696,-0.686889,-0.828628,0.223205


In [26]:
df["T"]["bar"]

lvl2,700,600,500,400
BK7,0.474865,-0.652418,0.766789,0.968327
F2,-1.100909,-1.048229,0.227380,-0.209260
SF6,-1.071696,-0.686889,-0.828628,0.223205


In [27]:
df.columns.levels

FrozenList([['T', 'nd', 'vd'], ['bar', 'foo'], [400, 500, 600, 700]])

In [28]:
df.columns.get_level_values(2)

Float64Index([nan, nan, 700.0, 600.0, 500.0, 400.0, 700.0, 600.0, 500.0,
              400.0],
             dtype='float64', name='lvl2')

# Toy version of 2 level MultiIndex

In [29]:
data_header=['nt','nC','nd','nF','A0','A1','A2','A3','T700','T600','T500','T400']

In [30]:
category=['rind','rind','rind','rind','coefs','coefs','coefs','coefs','trans','trans','trans','trans']

In [31]:
mind = pd.MultiIndex.from_arrays([category, data_header])

In [32]:
category, data_header

(['rind',
  'rind',
  'rind',
  'rind',
  'coefs',
  'coefs',
  'coefs',
  'coefs',
  'trans',
  'trans',
  'trans',
  'trans'],
 ['nt',
  'nC',
  'nd',
  'nF',
  'A0',
  'A1',
  'A2',
  'A3',
  'T700',
  'T600',
  'T500',
  'T400'])

In [33]:
data_df=pd.DataFrame(np.array([[1,2,3,4,5,6,7,8,9,10,11,12],[13,14,15,16,17,18,19,20,21,22,23,24]]),
                     index=['ABC','DEF'], columns=mind)

In [34]:
data_df

rind             coefs             trans               
      nt  nC  nd  nF    A0  A1  A2  A3  T700 T600 T500 T400
ABC    1   2   3   4     5   6   7   8     9   10   11   12
DEF   13  14  15  16    17  18  19  20    21   22   23   24

In [35]:
data_df['rind']

,nt,nC,nd,nF
ABC,1,2,3,4
DEF,13,14,15,16


In [36]:
data_df.loc['ABC']

rind   nt       1
       nC       2
       nd       3
       nF       4
coefs  A0       5
       A1       6
       A2       7
       A3       8
trans  T700     9
       T600    10
       T500    11
       T400    12
Name: ABC, dtype: int64

In [37]:
data_df.loc['ABC']['rind']

nt    1
nC    2
nd    3
nF    4
Name: ABC, dtype: int64

In [38]:
data_df.loc['ABC']['rind']['nC']

2

In [39]:
mind.get_loc('rind')

array([ True,  True,  True,  True, False, False, False, False, False,
       False, False, False])

In [40]:
mind.get_level_values(0)

Index(['rind', 'rind', 'rind', 'rind', 'coefs', 'coefs', 'coefs', 'coefs',
       'trans', 'trans', 'trans', 'trans'],
      dtype='object')

# Build Multi-Index for Hoya

In [41]:
hoya_df=xl2df('HOYA.xlsx')

In [42]:
hoya_df[:5].T['NL':'PE']

,0,1,2,3,4
NL,NaN,Internal Transmittance (2mmt),(2mmt),2500,NaN
NM,NaN,NaN,NaN,2400.0,NaN
NN,NaN,NaN,NaN,2200.0,NaN
NO,NaN,NaN,NaN,2000.0,NaN
NP,NaN,NaN,NaN,1800.0,NaN
NQ,NaN,NaN,NaN,1600.0,NaN
NR,NaN,NaN,NaN,1550.0,0.998
NS,NaN,NaN,NaN,1500.0,0.998
NT,NaN,NaN,NaN,1400.0,0.994
NU,NaN,NaN,NaN,1300.0,0.999


In [43]:
hoya_df

,A,B,C,D,E,F,G,H,I,J,...,SR,SS,ST,SU,SV,SW,SX,SY,SZ,TA
0,Classification,Idx,Glass Type,Code,Refractive\nIndex,Abbe-\nnumber,Dispersion,Refractive \nIndex,Abbe-\nnumber,Dispersion,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Remarks,Update
1,NaN,NaN,NaN,NaN,nd,νd,nF-nC,ne,νe,nF'-nC',...,OHARA,NaN,HIKARI,NaN,SUMITA,NaN,CDGM,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,587.56,NaN,NaN,546.07,NaN,NaN,...,Glass Type,Code,Glass Type,Code,Glass Type,Code,Glass Type,Code,NaN,NaN
4,NaN,1,FC5,487-704,1.48749,70.45,0.00692,1.48914,70.24,0.006964,...,S-FSL 5,487-702,J-FK5,487-703,K-FK5,487-704,H-QK3L,487-704,NaN,20170401
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193,Ref.,189,M-NBF1,743-493,1.7433,49.33,0.015069,1.74689,49.07,0.015221,...,L-LAM60,743-493,Q-LAF010S,743-493,NaN,NaN,D-LaF53,743-493,NaN,20170401
194,Ref.,190,MP-NBF1,743-493,1.7433,49.33,0.015069,1.74689,49.07,0.015221,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20170401
195,Ref.,191,MC-NBF1,743-493,1.7433,49.33,0.015069,1.74689,49.07,0.015221,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20170401
196,Ref.,192,M-TAF1,773-495,1.7725,49.47,0.015617,1.77622,49.2,0.015776,...,NaN,NaN,NaN,NaN,(K-LaFK50),(772-500),NaN,NaN,NaN,20170401


In [44]:
hoya_df.iloc[0]

A        Classification
B                   Idx
C            Glass　Type
D                  Code
E     Refractive\nIndex
            ...        
SW                  NaN
SX                  NaN
SY                  NaN
SZ              Remarks
TA               Update
Name: 0, Length: 521, dtype: object

In [45]:
data_header0, df_categories0 = fill_in_nans(hoya_df.iloc[0])

In [46]:
df_categories0

{'Classification': ((0, 'A'), (1, 'A')),
 'Idx': ((1, 'B'), (2, 'B')),
 'Glass\u3000Type': ((2, 'C'), (3, 'C')),
 'Code': ((3, 'D'), (4, 'D')),
 'Refractive\nIndex': ((4, 'E'), (5, 'E')),
 'Abbe-\nnumber': ((8, 'I'), (9, 'I')),
 'Dispersion': ((9, 'J'), (10, 'J')),
 'Refractive \nIndex': ((7, 'H'), (8, 'H')),
 'Refractive Index': ((10, 'K'), (27, 'AA')),
 'blank': ((27, 'AB'), (28, 'AB')),
 'Constants of Dispersion Formula ': ((28, 'AC'), (40, 'AN')),
 'Partial Dispersions': ((40, 'AO'), (49, 'AW')),
 'Partial Dispersion Ratio ': ((49, 'AX'), (69, 'BQ')),
 'Deviation of Relative Partial Dispersions': ((69, 'BR'), (74, 'BV')),
 'Chemical Properties': ((74, 'BW'), (81, 'CC')),
 'Thermal Properties': ((81, 'CD'), (109, 'DE')),
 'Mechanical  Properties': ((109, 'DF'), (116, 'DL')),
 'Temperature Coefficient of Refractive Index': ((116, 'DM'), (122, 'DR')),
 'Temperature Coefficient of Refractive Index  nh  404.66   (×10-6/℃)': ((122,
   'DS'),
  (142, 'EL')),
 'Temperature Coefficient of R

In [47]:
data_header1, df_categories1 = fill_in_nans(hoya_df.iloc[1])

In [48]:
df_categories1

{'nd': ((20, 'U'), (21, 'U')),
 'νd': ((5, 'F'), (6, 'F')),
 'nF-nC': ((6, 'G'), (7, 'G')),
 'ne': ((21, 'V'), (22, 'V')),
 'νe': ((8, 'I'), (9, 'I')),
 "nF'-nC'": ((9, 'J'), (10, 'J')),
 'n1529.6': ((10, 'K'), (11, 'K')),
 'n1128.64': ((11, 'L'), (12, 'L')),
 'nt': ((12, 'M'), (13, 'M')),
 'ns': ((13, 'N'), (14, 'N')),
 "nA'": ((14, 'O'), (15, 'O')),
 'nr': ((15, 'P'), (16, 'P')),
 'nC': ((16, 'Q'), (17, 'Q')),
 "nC'": ((17, 'R'), (18, 'R')),
 'nHe-Ne': ((18, 'S'), (19, 'S')),
 'nD': ((19, 'T'), (20, 'T')),
 'nF': ((22, 'W'), (23, 'W')),
 "nF'": ((23, 'X'), (24, 'X')),
 'ng': ((24, 'Y'), (25, 'Y')),
 'nh': ((25, 'Z'), (26, 'Z')),
 'ni': ((26, 'AA'), (28, 'AB')),
 'A0': ((28, 'AC'), (29, 'AC')),
 'A0pow': ((29, 'AD'), (30, 'AD')),
 'A1': ((30, 'AE'), (31, 'AE')),
 'A1pow': ((31, 'AF'), (32, 'AF')),
 'A2': ((32, 'AG'), (33, 'AG')),
 'A2pow': ((33, 'AH'), (34, 'AH')),
 'A3': ((34, 'AI'), (35, 'AI')),
 'A3pow': ((35, 'AJ'), (36, 'AJ')),
 'A4': ((36, 'AK'), (37, 'AK')),
 'A4pow': ((37, 'AL

In [49]:
df_categories1['Internal Transmittance (2mmt)']

((375, 'NL'), (419, 'PC'))

In [50]:
data_header2, df_categories2 = fill_in_nans(hoya_df.iloc[2])

In [51]:
df_categories2

{'α (-40/-30)': ((86, 'CI'), (87, 'CI')),
 'α (-30/-20)': ((87, 'CJ'), (88, 'CJ')),
 'α (-20/-10)': ((88, 'CK'), (89, 'CK')),
 'α (-10/0)': ((89, 'CL'), (90, 'CL')),
 'α (0/10)': ((90, 'CM'), (91, 'CM')),
 'α (10/20)': ((91, 'CN'), (92, 'CN')),
 'α (20/30)': ((92, 'CO'), (93, 'CO')),
 'α (30/40)': ((93, 'CP'), (94, 'CP')),
 'α (40/50)': ((94, 'CQ'), (95, 'CQ')),
 'α (50/60)': ((95, 'CR'), (96, 'CR')),
 'α (60/70)': ((96, 'CS'), (97, 'CS')),
 'α (70/80)': ((97, 'CT'), (98, 'CT')),
 'α (80/90)': ((98, 'CU'), (99, 'CU')),
 'α (90/100)': ((99, 'CV'), (100, 'CV')),
 'α (100/110)': ((100, 'CW'), (101, 'CW')),
 'α (110/120)': ((101, 'CX'), (102, 'CX')),
 'α (120/130)': ((102, 'CY'), (103, 'CY')),
 'α (130/140)': ((103, 'CZ'), (104, 'CZ')),
 'α (140/150)': ((104, 'DA'), (105, 'DA')),
 'α (-30/70)': ((105, 'DB'), (106, 'DB')),
 'α (100/300)': ((106, 'DC'), (116, 'DL')),
 'D0': ((116, 'DM'), (117, 'DM')),
 'D1': ((117, 'DN'), (118, 'DN')),
 'D2': ((118, 'DO'), (119, 'DO')),
 'E0': ((119, 'DP'), 

In [52]:
data_header3, df_categories3 = fill_in_nans(hoya_df.iloc[3])

In [53]:
df_categories3

{587.56: ((20, 'U'), (21, 'U')),
 546.07: ((21, 'V'), (22, 'V')),
 1529.6: ((10, 'K'), (11, 'K')),
 1128.64: ((11, 'L'), (12, 'L')),
 1013.98: ((12, 'M'), (13, 'M')),
 852.11: ((13, 'N'), (14, 'N')),
 768.19: ((14, 'O'), (15, 'O')),
 706.52: ((15, 'P'), (16, 'P')),
 656.27: ((16, 'Q'), (17, 'Q')),
 643.85: ((17, 'R'), (18, 'R')),
 632.8: ((18, 'S'), (19, 'S')),
 589.29: ((19, 'T'), (20, 'T')),
 486.13: ((22, 'W'), (23, 'W')),
 479.99: ((23, 'X'), (24, 'X')),
 435.84: ((24, 'Y'), (25, 'Y')),
 404.66: ((25, 'Z'), (26, 'Z')),
 365.01: ((26, 'AA'), (74, 'BV')),
 'DW': ((74, 'BW'), (75, 'BW')),
 'DA': ((75, 'BX'), (76, 'BX')),
 'Tblue': ((76, 'BY'), (77, 'BY')),
 'DNaOH': ((77, 'BZ'), (78, 'BZ')),
 'DSTPP': ((78, 'CA'), (79, 'CA')),
 'D0': ((79, 'CB'), (80, 'CB')),
 'DH': ((80, 'CC'), (81, 'CC')),
 'Tg': ((81, 'CD'), (82, 'CD')),
 'Ts': ((82, 'CE'), (83, 'CE')),
 'T1014.5': ((83, 'CF'), (84, 'CF')),
 'T1013': ((84, 'CG'), (85, 'CG')),
 'T107.6': ((85, 'CH'), (107, 'DC')),
 'λ': ((107, 'DD')

In [54]:
span2 = df_categories1['Internal Transmittance (2mmt)']
span5 = df_categories1['Internal Transmittance (5mmt)']
span10 = df_categories1['Internal Transmittance (10mmt)']
span2, span5, span10

(((375, 'NL'), (419, 'PC')),
 ((419, 'PD'), (463, 'QU')),
 ((463, 'QV'), (507, 'SM')))

In [55]:
data_header0[span2[0][0]:span2[1][0]]

['Spectral Transmittance ',
 'Spectral Transmittance ',
 'Spectral Transmittance ',
 'Spectral Transmittance ',
 'Spectral Transmittance ',
 'Spectral Transmittance ',
 'Spectral Transmittance ',
 'Spectral Transmittance ',
 'Spectral Transmittance ',
 'Spectral Transmittance ',
 'Spectral Transmittance ',
 'Spectral Transmittance ',
 'Spectral Transmittance ',
 'Spectral Transmittance ',
 'Spectral Transmittance ',
 'Spectral Transmittance ',
 'Spectral Transmittance ',
 'Spectral Transmittance ',
 'Spectral Transmittance ',
 'Spectral Transmittance ',
 'Spectral Transmittance ',
 'Spectral Transmittance ',
 'Spectral Transmittance ',
 'Spectral Transmittance ',
 'Spectral Transmittance ',
 'Spectral Transmittance ',
 'Spectral Transmittance ',
 'Spectral Transmittance ',
 'Spectral Transmittance ',
 'Spectral Transmittance ',
 'Spectral Transmittance ',
 'Spectral Transmittance ',
 'Spectral Transmittance ',
 'Spectral Transmittance ',
 'Spectral Transmittance ',
 'Spectral Transmitt

In [56]:
data_header1[span2[0][0]:span2[1][0]]

['Internal Transmittance (2mmt)',
 'Internal Transmittance (2mmt)',
 'Internal Transmittance (2mmt)',
 'Internal Transmittance (2mmt)',
 'Internal Transmittance (2mmt)',
 'Internal Transmittance (2mmt)',
 'Internal Transmittance (2mmt)',
 'Internal Transmittance (2mmt)',
 'Internal Transmittance (2mmt)',
 'Internal Transmittance (2mmt)',
 'Internal Transmittance (2mmt)',
 'Internal Transmittance (2mmt)',
 'Internal Transmittance (2mmt)',
 'Internal Transmittance (2mmt)',
 'Internal Transmittance (2mmt)',
 'Internal Transmittance (2mmt)',
 'Internal Transmittance (2mmt)',
 'Internal Transmittance (2mmt)',
 'Internal Transmittance (2mmt)',
 'Internal Transmittance (2mmt)',
 'Internal Transmittance (2mmt)',
 'Internal Transmittance (2mmt)',
 'Internal Transmittance (2mmt)',
 'Internal Transmittance (2mmt)',
 'Internal Transmittance (2mmt)',
 'Internal Transmittance (2mmt)',
 'Internal Transmittance (2mmt)',
 'Internal Transmittance (2mmt)',
 'Internal Transmittance (2mmt)',
 'Internal Tra

In [57]:
data_header3[span2[0][0]:span2[1][0]]

[2500,
 2400.0,
 2200.0,
 2000.0,
 1800.0,
 1600.0,
 1550.0,
 1500.0,
 1400.0,
 1300.0,
 1200.0,
 1100,
 1060.0,
 1050.0,
 1000.0,
 950.0,
 900.0,
 850.0,
 830.0,
 800.0,
 780.0,
 750.0,
 700,
 650.0,
 600.0,
 550.0,
 500.0,
 480.0,
 460.0,
 440.0,
 420.0,
 400.0,
 390.0,
 380,
 370.0,
 360.0,
 350.0,
 340.0,
 330.0,
 320.0,
 310.0,
 300.0,
 290.0,
 280.0]

In [58]:
def modify_data_headers(hoya_df, promotion_list, num_rows):
    data_headers = []
    categories = []
    for i in range(num_rows):
        data_header, df_categories = fill_in_nans(hoya_df.iloc[i])
        data_headers.append(data_header)
        categories.append(df_categories)
    for key in promotion_list:
        span = categories[1][key]
        i1 = span[0][0]
        ik = span[1][0]
        data_headers[0][i1:ik] = data_headers[1][i1:ik]
        data_headers[1][i1:ik] = data_headers[3][i1:ik]
    mindx = pd.MultiIndex.from_arrays([data_headers[0][3:], data_headers[1][3:]])
    return mindx, data_headers, categories

In [59]:
promotion_list = [
    'Internal Transmittance (2mmt)',
    'Internal Transmittance (5mmt)',
    'Internal Transmittance (10mmt)',
    'Chromaticity',
]
mindx, data_headers, categories = modify_data_headers(hoya_df, promotion_list, 4)

In [60]:
df_categories=[c for c in hoya_df.iloc[2] if not pd.isna(c)]

In [61]:
df_categories = []
data_header0 = []
for c in hoya_df.iloc[0]:
    if not pd.isna(c):
        c_last = c
        df_categories.append(c)
        data_header0.append(c)
    else:
        data_header0.append(c_last)

In [62]:
len(df_categories)

39

In [63]:
df_categories

['Classification',
 'Idx',
 'Glass\u3000Type',
 'Code',
 'Refractive\nIndex',
 'Abbe-\nnumber',
 'Dispersion',
 'Refractive \nIndex',
 'Abbe-\nnumber',
 'Dispersion',
 'Refractive Index',
 'blank',
 'Constants of Dispersion Formula ',
 'Partial Dispersions',
 'Partial Dispersion Ratio ',
 'Deviation of Relative Partial Dispersions',
 'Chemical Properties',
 'Thermal Properties',
 'Mechanical  Properties',
 'Temperature Coefficient of Refractive Index',
 'Temperature Coefficient of Refractive Index  nh  404.66   (×10-6/℃)',
 'Temperature Coefficient of Refractive Index  ng  435.84   (×10-6/℃)',
 "Temperature Coefficient of Refractive Index  nF' 479.99   (×10-6/℃)",
 'Temperature Coefficient of Refractive Index  nF  486.13   (×10-6/℃)',
 'Temperature Coefficient of Refractive Index  ne  546.07   (×10-6/℃)',
 'Temperature Coefficient of Refractive Index  nd  587.56   (×10-6/℃)',
 'Temperature Coefficient of Refractive Index  nHe-Ne  632.8   (×10-6/℃)',
 "Temperature Coefficient of Refract

In [64]:
data_header1 = hoya_df.iloc[1]
data_header2 = hoya_df.iloc[2]
data_header3 = hoya_df.iloc[3]

In [65]:
data_header1['QV']

'Internal Transmittance (10mmt)'

In [66]:
data_header1.array[463]

'Internal Transmittance (10mmt)'

In [67]:
for i, iv in enumerate(data_header1.items()):
    index, value = iv
    print(f"Index : {index}, Idx : {i}, Value : {value}")
#    str.index('Internal Transmittance (2mmt)')

Index : A, Idx : 0, Value : nan
Index : B, Idx : 1, Value : nan
Index : C, Idx : 2, Value : nan
Index : D, Idx : 3, Value : nan
Index : E, Idx : 4, Value : nd
Index : F, Idx : 5, Value : νd
Index : G, Idx : 6, Value : nF-nC
Index : H, Idx : 7, Value : ne
Index : I, Idx : 8, Value : νe
Index : J, Idx : 9, Value : nF'-nC'
Index : K, Idx : 10, Value : n1529.6
Index : L, Idx : 11, Value : n1128.64
Index : M, Idx : 12, Value : nt
Index : N, Idx : 13, Value : ns
Index : O, Idx : 14, Value : nA'
Index : P, Idx : 15, Value : nr
Index : Q, Idx : 16, Value : nC
Index : R, Idx : 17, Value : nC'
Index : S, Idx : 18, Value : nHe-Ne
Index : T, Idx : 19, Value : nD
Index : U, Idx : 20, Value : nd
Index : V, Idx : 21, Value : ne
Index : W, Idx : 22, Value : nF
Index : X, Idx : 23, Value : nF'
Index : Y, Idx : 24, Value : ng
Index : Z, Idx : 25, Value : nh
Index : AA, Idx : 26, Value : ni
Index : AB, Idx : 27, Value : nan
Index : AC, Idx : 28, Value : A0
Index : AD, Idx : 29, Value : A0pow
Index : AE, 

In [68]:
for index, value in data_header2.items():
    print(f"Index : {index}, Value : {value}")
#    str.index('Internal Transmittance (2mmt)')

Index : A, Value : nan
Index : B, Value : nan
Index : C, Value : nan
Index : D, Value : nan
Index : E, Value : nan
Index : F, Value : nan
Index : G, Value : nan
Index : H, Value : nan
Index : I, Value : nan
Index : J, Value : nan
Index : K, Value : nan
Index : L, Value : nan
Index : M, Value : nan
Index : N, Value : nan
Index : O, Value : nan
Index : P, Value : nan
Index : Q, Value : nan
Index : R, Value : nan
Index : S, Value : nan
Index : T, Value : nan
Index : U, Value : nan
Index : V, Value : nan
Index : W, Value : nan
Index : X, Value : nan
Index : Y, Value : nan
Index : Z, Value : nan
Index : AA, Value : nan
Index : AB, Value : nan
Index : AC, Value : nan
Index : AD, Value : nan
Index : AE, Value : nan
Index : AF, Value : nan
Index : AG, Value : nan
Index : AH, Value : nan
Index : AI, Value : nan
Index : AJ, Value : nan
Index : AK, Value : nan
Index : AL, Value : nan
Index : AM, Value : nan
Index : AN, Value : nan
Index : AO, Value : nan
Index : AP, Value : nan
Index : AQ, Value 

In [69]:
for index, value in data_header3.items():
    print(f"Index : {index}, Value : {value}")
#    str.index('Internal Transmittance (2mmt)')

Index : A, Value : nan
Index : B, Value : nan
Index : C, Value : nan
Index : D, Value : nan
Index : E, Value : 587.56
Index : F, Value : nan
Index : G, Value : nan
Index : H, Value : 546.07
Index : I, Value : nan
Index : J, Value : nan
Index : K, Value : 1529.6
Index : L, Value : 1128.64
Index : M, Value : 1013.98
Index : N, Value : 852.11
Index : O, Value : 768.19
Index : P, Value : 706.52
Index : Q, Value : 656.27
Index : R, Value : 643.85
Index : S, Value : 632.8
Index : T, Value : 589.29
Index : U, Value : 587.56
Index : V, Value : 546.07
Index : W, Value : 486.13
Index : X, Value : 479.99
Index : Y, Value : 435.84
Index : Z, Value : 404.66
Index : AA, Value : 365.01
Index : AB, Value : nan
Index : AC, Value : nan
Index : AD, Value : nan
Index : AE, Value : nan
Index : AF, Value : nan
Index : AG, Value : nan
Index : AH, Value : nan
Index : AI, Value : nan
Index : AJ, Value : nan
Index : AK, Value : nan
Index : AL, Value : nan
Index : AM, Value : nan
Index : AN, Value : nan
Index : 

In [70]:
mindx = pd.MultiIndex.from_arrays([data_header0[3:], data_header1[3:]])

In [71]:
mindx

MultiIndex([(                       'Code',        nan),
            (          'Refractive\nIndex',       'nd'),
            (              'Abbe-\nnumber',       'νd'),
            (                 'Dispersion',    'nF-nC'),
            (         'Refractive \nIndex',       'ne'),
            (              'Abbe-\nnumber',       'νe'),
            (                 'Dispersion',  'nF'-nC''),
            (           'Refractive Index',  'n1529.6'),
            (           'Refractive Index', 'n1128.64'),
            (           'Refractive Index',       'nt'),
            ...
            ('Glass Cross Reference Index',    'OHARA'),
            ('Glass Cross Reference Index',        nan),
            ('Glass Cross Reference Index',   'HIKARI'),
            ('Glass Cross Reference Index',        nan),
            ('Glass Cross Reference Index',   'SUMITA'),
            ('Glass Cross Reference Index',        nan),
            ('Glass Cross Reference Index',     'CDGM'),
            ('G

In [72]:
args = 4, 197, 'D', 'TA'
kwargs = dict(
    name_col_offset='C',
    data_header=1,
    coef_str='A0',
    num_coefs=12,
    rindex_str='n1529.6',
    transmission_offset=460,
    num_wvls=44,
    transmission_wvl_shape=(3, None, 'QV', 'SM'),
    )
hoya = df2gcat(hoya_df, *args, **kwargs)

In [73]:
hoya

1,NaN,nd,νd,nF-nC,ne,νe,nF'-nC',n1529.6,n1128.64,nt,...,OHARA,NaN,HIKARI,NaN,SUMITA,NaN,CDGM,NaN,NaN,NaN
C,,,,,,,,,,,,,,,,,,,,,
FC5,487-704,1.48749,70.45,0.00692,1.48914,70.24,0.006964,1.47308,1.47774,1.47913,...,S-FSL 5,487-702,J-FK5,487-703,K-FK5,487-704,H-QK3L,487-704,NaN,20170401
FCD1,497-816,1.497,81.61,0.00609,1.49845,81.19,0.006139,1.48598,1.48907,1.49008,...,S-FPL51,497-816,J-FK01,497-817,NaN,NaN,H-FK61,497-816,NaN,20181012
FCD1B,497-816,1.4971,81.56,0.006095,1.49856,81.15,0.006144,1.48592,1.48912,1.49015,...,S-FPL51,497-816,J-FK01,497-817,NaN,NaN,H-FK61,497-816,NaN,20181012
FCD10A,459-902,1.4586,90.2,0.005084,1.45981,89.75,0.005123,1.44937,1.45194,1.45279,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20170401
FCD100,437-951,1.437,95.1,0.004595,1.4381,94.66,0.004628,1.42855,1.43094,1.43172,...,(S-FPL53),(439-950),NaN,NaN,NaN,NaN,NaN,NaN,NaN,20170401
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
M-NBF1,743-493,1.7433,49.33,0.015069,1.74689,49.07,0.015221,1.71791,1.72466,1.72693,...,L-LAM60,743-493,Q-LAF010S,743-493,NaN,NaN,D-LaF53,743-493,NaN,20170401
MP-NBF1,743-493,1.7433,49.33,0.015069,1.74689,49.07,0.015221,1.71791,1.72466,1.72693,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20170401
MC-NBF1,743-493,1.7433,49.33,0.015069,1.74689,49.07,0.015221,1.71791,1.72466,1.72693,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20170401


In [74]:
hoya.columns = mindx

In [75]:
hoya

Code Refractive\nIndex Abbe-\nnumber Dispersion  \
1            NaN                nd            νd      nF-nC   
C                                                             
FC5      487-704           1.48749         70.45    0.00692   
FCD1     497-816             1.497         81.61    0.00609   
FCD1B    497-816            1.4971         81.56   0.006095   
FCD10A   459-902            1.4586          90.2   0.005084   
FCD100   437-951             1.437          95.1   0.004595   
...          ...               ...           ...        ...   
M-NBF1   743-493            1.7433         49.33   0.015069   
MP-NBF1  743-493            1.7433         49.33   0.015069   
MC-NBF1  743-493            1.7433         49.33   0.015069   
M-TAF1   773-495            1.7725         49.47   0.015617   
MC-TAF1  773-495            1.7725         49.47   0.015617   

        Refractive \nIndex Abbe-\nnumber Dispersion Refractive Index           \
1                       ne            νe    nF'-nC'          n1529.6 n1128.64   
C                                                                               
FC5                1.48914         70.24   0.006964          1.47308  1.47774   
FCD1               1.49845         81.19   0.006139          1.48598  1.48907   
FCD1B              1.49856         81.15   0.006144          1.48592  1.48912   
FCD10A             1.45981         89.75   0.005123          1.44937  1.45194   
FCD100              1.4381         94.66   0.004628          1.42855  1.43094   
...                    ...           ...        ...              ...      ...   
M-NBF1             1.74689         49.07   0.015221          1.71791  1.72466   
MP-NBF1            1.74689         49.07   0.015221          1.71791  1.72466   
MC-NBF1            1.74689         49.07   0.015221          1.71791  1.72466   
M-TAF1             1.77622          49.2   0.015776          1.74719   1.7535   
MC-TAF1            1.77622          49.2   0.015776          1.74719   1.7535   

                  ... Glass Cross Reference Index                        \
1             nt  ...                       OHARA        NaN     HIKARI   
C                 ...                                                     
FC5      1.47913  ...                     S-FSL 5    487-702      J-FK5   
FCD1     1.49008  ...                     S-FPL51    497-816     J-FK01   
FCD1B    1.49015  ...                     S-FPL51    497-816     J-FK01   
FCD10A   1.45279  ...                         NaN        NaN        NaN   
FCD100   1.43172  ...                   (S-FPL53)  (439-950)        NaN   
...          ...  ...                         ...        ...        ...   
M-NBF1   1.72693  ...                     L-LAM60    743-493  Q-LAF010S   
MP-NBF1  1.72693  ...                         NaN        NaN        NaN   
MC-NBF1  1.72693  ...                         NaN        NaN        NaN   
M-TAF1   1.75571  ...                         NaN        NaN        NaN   
MC-TAF1  1.75571  ...                         NaN        NaN        NaN   

                                                          Remarks    Update  
1            NaN      SUMITA        NaN     CDGM      NaN     NaN       NaN  
C                                                                            
FC5      487-703       K-FK5    487-704   H-QK3L  487-704     NaN  20170401  
FCD1     497-817         NaN        NaN   H-FK61  497-816     NaN  20181012  
FCD1B    497-817         NaN        NaN   H-FK61  497-816     NaN  20181012  
FCD10A       NaN         NaN        NaN      NaN      NaN     NaN  20170401  
FCD100       NaN         NaN        NaN      NaN      NaN     NaN  20170401  
...          ...         ...        ...      ...      ...     ...       ...  
M-NBF1   743-493         NaN        NaN  D-LaF53  743-493     NaN  20170401  
MP-NBF1      NaN         NaN        NaN      NaN      NaN     NaN  20170401  
MC-NBF1      NaN         NaN        NaN      NaN      NaN     NaN  20170401  
M-TAF1 

In [76]:
len(hoya.loc['BSC7']['Constants of Dispersion Formula '])

12

In [77]:
hoya.loc['BSC7']['Refractive Index']

1
n1529.6     1.50123
n1128.64    1.50586
nt          1.50733
ns           1.5098
nA'         1.51143
nr          1.51289
nC          1.51432
nC'         1.51472
nHe-Ne      1.51509
nD          1.51673
nd           1.5168
ne          1.51872
nF          1.52237
nF'         1.52282
ng          1.52667
nh          1.53022
ni          1.53622
Name: BSC7, dtype: object

In [78]:
hoya_df.axes

[RangeIndex(start=0, stop=198, step=1),
 Index(['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J',
        ...
        'SR', 'SS', 'ST', 'SU', 'SV', 'SW', 'SX', 'SY', 'SZ', 'TA'],
       dtype='object', length=521)]

In [79]:
hoya_df.shape

(198, 521)

In [80]:
hoya.loc['BSC7']['Spectral Transmittance ']

1
Coloration Code                           325
NaN                                       NaN
NaN                                       285
Coloration of Internal Transmittance      319
NaN                                       283
                                        ...  
NaN                                     0.396
NaN                                     0.151
NaN                                     0.027
Chromaticity                              NaN
NaN                                       NaN
Name: BSC7, Length: 141, dtype: object

In [81]:
hoya.loc['FCD1']['Internal Transmittance (10mmt)'][340.]

KeyError: 'Internal Transmittance (10mmt)'

In [ ]:
hoya.loc['BSC7']['Update']

In [ ]:
data_headers[0][-10:]

In [ ]:
data_headers[1][-10:]

In [ ]:
hoya.loc['FCD1']['Refractive Index']['nt']

In [ ]:
hoya.loc['FCD1']['Mechanical  Properties']

In [ ]:
hoya.loc['FCD1']['Mechanical  Properties']['Knoop Hardness']

In [ ]:
hoya.loc['FCD1']['Spec. Gravity']